In [102]:
import pandas as pd
import sqlite3 as sql

In [103]:
excel_file = ['temp.xlsx', 'Prepaid_NE1_E_KYC_Jan_20_B_Quer']
out_db = 'North East - Airtel.db'
circle = 11
operator = 2

In [104]:
! dir

 Volume in drive C has no label.
 Volume Serial Number is 6A57-0033

 Directory of C:\Users\Ichimichi Laribok\Documents\neon

01-09-2020  14:39    <DIR>          .
01-09-2020  14:39    <DIR>          ..
01-09-2020  11:53    <DIR>          .ipynb_checkpoints
01-09-2020  11:40        76,788,401 Airtel_Prepaid_NE1_E_KYC_Jan_20_B.xlsx
31-08-2020  08:29           553,211 batch2016.xlsx
01-09-2020  14:12    <DIR>          data
01-09-2020  14:39            25,412 excel-to-sql.ipynb
01-09-2020  12:08            51,295 temp.xlsx
               4 File(s)     77,418,319 bytes
               4 Dir(s)  151,299,129,344 bytes free


In [105]:
df = pd.read_excel(
    excel_file[0], 
    sheet_name=excel_file[1],
    header=0)

In [106]:
df.tail()

,Telephone_Number,Name_of_Subscriber,Permanent__House_No__Flat_No,Permanent__Street_Address_Name,Permanent__Locality,Permanent_City,Permanent_District,Permanent_Local_State_UT,Permanent_l_Postal_Code,Circle,SIM_Activation_Date
495,7085113559,Fanindra Biswas,5/38,Ward 4,"Barpathari , Barpathari Panchayet, Sub Div ,Be...",South Tripura,NaN,Tripura,799155,NE1,2/10/2016
496,7085113651,Laxmi Debnath,Ward No-03,NaN,"Vill-R K Ganj,Block-Bokafa,Sub-Santir Bazar,Ra...",South Tripura,NaN,Tripura,799144,NE1,2/10/2016
497,7085113661,Kyaching Mog,MOHA MONI ROAD WARD 9,NaN,"SANTIR BAZAR,Santirbazar,",South Tripura,NaN,Tripura,799144,NE1,2/10/2016
498,7085113740,Praban Debbarma,"Balurbandh Baikantha Pur,Fari Bari",NaN,NaN,West Tripura,NaN,Tripura,799212,NE1,12/18/2015
499,7085113828,Debabrata Paul,06/281,Purba Para Ward No-06,"P/O-Santirbazar,Suv Div-Santirbazar,Santirbaza...",South Tripura,NaN,Tripura,799144,NE1,2/22/2016


In [107]:
df['Circle'] = circle
df['Operator'] = operator

In [108]:
df = df.fillna('')
df['Permanent_l_Postal_Code'] = df['Permanent_l_Postal_Code'].apply(str)
df['Telephone_Number'] = df['Telephone_Number'].apply(str)

In [109]:
df['Address'] = df['Permanent__House_No__Flat_No']+', '+df['Permanent__Street_Address_Name']+', '+df['Permanent__Locality']+', '+df['Permanent_City']+', '+df['Permanent_Local_State_UT']+', '+df['Permanent_l_Postal_Code']

In [110]:
df = df.drop(['Permanent_District',
              'Permanent__House_No__Flat_No',
              'Permanent__Street_Address_Name',
              'Permanent__Locality', 
              'Permanent_City', 
              'Permanent_Local_State_UT', 
              'Permanent_l_Postal_Code'], axis=1)

In [111]:
df = df.rename(columns=
               {
                   "Telephone_Number": "Number", 
                   "Name_of_Subscriber": "Name", 
                   "SIM_Activation_Date": "Date",
               })

In [112]:
df['misc'] = df['Number'].str[:5]


misc = list(set([str(item)[:5] for item in list(df.Number.unique())]))

In [113]:
df.tail()

,Number,Name,Circle,Date,Operator,Address,misc
495,7085113559,Fanindra Biswas,11,2/10/2016,2,"5/38, Ward 4, Barpathari , Barpathari Panchaye...",70851
496,7085113651,Laxmi Debnath,11,2/10/2016,2,"Ward No-03, , Vill-R K Ganj,Block-Bokafa,Sub-S...",70851
497,7085113661,Kyaching Mog,11,2/10/2016,2,"MOHA MONI ROAD WARD 9, , SANTIR BAZAR,Santirba...",70851
498,7085113740,Praban Debbarma,11,12/18/2015,2,"Balurbandh Baikantha Pur,Fari Bari, , , West T...",70851
499,7085113828,Debabrata Paul,11,2/22/2016,2,"06/281, Purba Para Ward No-06, P/O-Santirbazar...",70851


In [114]:
db_conn = sql.connect("./data/"+out_db)
c = db_conn.cursor()

In [117]:
for m in misc:
    c.execute(
        """
        DROP TABLE [{}];
        """.format(m)
    )

c.execute(
    """
    DROP TABLE [android_metadata];
    """
    )

OperationalError: no such table: android_metadata

In [118]:
for m in misc:
    c.execute(
        """
        CREATE TABLE [{}] (
            [ID] INTEGER PRIMARY KEY AUTOINCREMENT,
            [Number] CHAR,
            [Name] CHAR,
            [Address] CHAR,
            [Date] DATE,
            [Circle] INT,
            [Operator] INT
            );
         """.format(m)
    )

In [119]:
df_grouped = df.groupby(df.misc)

for m in misc:
    df_test = df_grouped.get_group(m)
    df_test = df_test.drop(['misc'], axis=1)
    df_test.to_sql(m, db_conn, if_exists='append', index=False)

In [120]:
c.execute(
    """
    CREATE TABLE [android_metadata] (
        [locale] TEXT DEFAULT 'en_US'
    );
    """
)

In [121]:
c.execute(
    """
    INSERT INTO [android_metadata] DEFAULT VALUES;
    """
)

In [122]:
pd.read_sql('SELECT * FROM [70851]', db_conn)

,ID,Number,Name,Address,Date,Circle,Operator
0,1,7085100204,Thakurmani Nath,"ward 7, , West para,Jubarajnagar,, North Tripu...",1/28/2016,11,2
1,2,7085100265,Rita Nath,"20, W-1, TILTHAI,Purba Tilthai,, North Tripura...",2/9/2016,11,2
2,3,7085100447,Sujan Chakraborty,"WARD NO-2, , RAMNAGAR,Ramnagar,NERA-SARADA SPU...",2/2/2016,11,2
3,4,7085100766,Jisaia Debbarma,"P T C Quarter, , Narsingarh,NARSINGARH,, West ...",11/18/2015,11,2
4,5,7085100790,Nitya Gopal Das,"Narsingarh, , Narsingarh Vivekananda Pally,Aga...",12/14/2015,11,2
...,...,...,...,...,...,...,...
418,419,7085113559,Fanindra Biswas,"5/38, Ward 4, Barpathari , Barpathari Panchaye...",2/10/2016,11,2
419,420,7085113651,Laxmi Debnath,"Ward No-03, , Vill-R K Ganj,Block-Bokafa,Sub-S...",2/10/2016,11,2
420,421,7085113661,Kyaching Mog,"MOHA MONI ROAD WARD 9, , SANTIR BAZAR,Santirba...",2/10/2016,11,2
421,422,7085113740,Praban Debbarma,"Balurbandh Baikantha Pur,Fari Bari, , , West T...",12/18/2015,11,2


In [123]:
pd.read_sql('SELECT * FROM [android_metadata]', db_conn)

,locale
0,en_US
